# Tutorial for Potential Model

This file is meant to demonstrate the use of gstlearn for Potential Model. It is exploited on an example in 1-D.

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

Setting some global variables

In [ ]:
# Define the Space Dimension
ndim = 1
gl.ASpaceObject_defineDefaultSpace(gl.ESpaceType.SPACE_RN, ndim)

Defining the data points used as controls

The set of iso-potential data points. Each datum is characterized by its coordinate and an indicator of the set to which it belongs.

In [ ]:
tabiso = [30., 1.,
         80., 1.,
         40., 2.,
         50., 2.]
dbiso = gl.Db.createFromSamples(4, gl.ELoadBy.SAMPLE, tabiso, ["x", "iso"], ["x1", "layer"])

The Gradient information

In [ ]:
tabgrd = [0., 1.]
dbgrd = gl.Db.createFromSamples(1, gl.ELoadBy.SAMPLE, tabgrd, ["x", "g"], ["x1", "g1"])

The &6D grid covering the field of estimation

In [ ]:
nx = [100]
grid = gl.DbGrid.create(nx)

The Model is composed of a Gaussian Covariance with a scale 20. The model must be turned into a Gradient model.

In [ ]:
range = gl.CovAniso.scale2range(gl.ECov.GAUSSIAN, 20.)
model = gl.Model.createFromParam(gl.ECov.GAUSSIAN, range)
model.switchToGradient()

The Neighborhood (although parametrized) should be Unique

In [ ]:
neighU = gl.NeighUnique.create(ndim)

## Launching the Estimation

We launch the calculation of the Potential on the nodes of the grid. The use of OptDbg statement is maintained for spying the process when estimating a target node in particular (-1 stands for no control).

In [ ]:
gl.OptDbg.setReference(-1)
gl.potential_kriging(dbiso, dbgrd, None, grid, model, neighU, 
                     flag_pot=True, flag_grad=True, flag_save_data=True, verbose=True)
gl.OptDbg.setReference(-1)

In order to overlay the information from the Iso-Potential file onto the Potential on the Grid represented as a section (vertical axis is the Potential axis), we must modify the locator of 'dbiso' (the Potential becomes the second coordinate)

In [ ]:
dbiso.setLocator("Potential",gl.ELoc.X,1)
dbiso

In [ ]:
ax = gp.grid1D(grid,figsize=[10,6])
ax = gp.point(dbiso, color_name="iso", ax=ax, aspect=None, size=100, flagColorBar=False)
plt.hlines(dbiso.getColumn("Potential"),0.,100.)